In [17]:
run quickstart.py

In [18]:
from gbt_scraper import *
import time
import copy

In [19]:
#get credentials and calendar list from Google Calendar
credentials = get_credentials()
http = credentials.authorize(httplib2.Http())
service = discovery.build('calendar', 'v3', http=http)
page_token = None
BL_CAL = 'BL_Obs'  #Choose to update calendar with this name
while True:
  calendar_list = service.calendarList().list(pageToken=page_token).execute()
  for calendar_list_entry in calendar_list['items']: #hti
    print calendar_list_entry['summary'] #hti
    if calendar_list_entry['summary'] == BL_CAL:
      cal = calendar_list_entry# ['items']
      print 'it worked'
      break
  page_token = calendar_list.get('nextPageToken')
  if not page_token:
    break

isabelangelo@berkeley.edu
BL_Obs
it worked


In [24]:
#scrape and store schedule from GBT website
scraper = GBTScraper(date=time.strftime("%m/%d/%Y"), days = 30) #store for next 30 days
scraper.make(t = 'utc')
events = scraper.btlpSchedule
#print events

In [50]:
def get_events(existing_events = []):
#	get list of existing events on google calendar
#	eventlist: list stores information about each event on calendar
#	returns: list containing start times of events on calendar
#    now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
    now = datetime.datetime.utcnow() # 'Z' indicates UTC time
    then= datetime.datetime.utcnow()-datetime.timedelta(7)
    #^^^assumes pulling from calendar in UTC^^^
    #maybe we can just subtract so that it starts from somewhere that
    #is always in the past?
    #should be fine if we account for existing events later
    t = then.isoformat() + 'Z'
    eventsResult = service.events().list(
        calendarId=cal['id'], timeMin=t, maxResults=100, singleEvents=True,
        orderBy='startTime', timeZone = 'Etc/UTC').execute()
    #or maybe do this in UTC time? this makes more sense
    events = eventsResult.get('items', [])

    if not events:
        print('No upcoming events found.')
    for event in events:
        #print event['summary']
        start = event['start'].get('dateTime', event['start'].get('date'))
        print start
        existing_events.append(start)

In [51]:
existing_events = []
get_events(existing_events)

2016-10-20T06:30:00Z
2016-10-21T04:30:00Z
2016-10-22T00:15:00Z
2016-10-22T18:15:00Z
2016-10-23T15:15:00Z
2016-10-23T18:30:00Z
2016-10-24T15:15:00Z
2016-10-24T17:00:00Z
2016-10-25T17:00:00Z
2016-10-26T19:00:00Z
2016-10-28T17:45:00Z
2016-10-28T20:45:00Z
2016-11-04
2016-11-04T04:00:00Z
